In [3]:
import pandas as pd, numpy as np

train = pd.read_csv(r'D:\Resume Projects\Backpack Prediction Challenge\train.csv')
print("Train shape",train.shape)
train_extra = pd.read_csv(r'D:\Resume Projects\Backpack Prediction Challenge\test.csv')
print("Extra Train shape",train_extra.shape)
train = pd.concat([train,train_extra],axis=0,ignore_index=True)
print("Combined Train shape",train.shape)
train.head()

Train shape (300000, 11)
Extra Train shape (200000, 10)
Combined Train shape (500000, 11)


,id,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg),Price
0,0,Jansport,Leather,Medium,7.0,Yes,No,Tote,Black,11.611723,112.15875
1,1,Jansport,Canvas,Small,10.0,Yes,Yes,Messenger,Green,27.078537,68.88056
2,2,Under Armour,Leather,Small,2.0,Yes,No,Messenger,Red,16.643760,39.17320
3,3,Nike,Nylon,Small,8.0,Yes,No,Messenger,Green,12.937220,80.60793
4,4,Adidas,Canvas,Medium,1.0,Yes,Yes,Messenger,Green,17.749338,86.02312


Baseline 1 - Train Mean - CV 38.93, LB 39.16

In [4]:
train_mean = train.Price.mean()
train['pred'] = train_mean
s = np.sqrt(np.mean( (train.Price-train.pred)**2.0 ) )
print(f"Validation RMSE using Train Mean = {s}")

Validation RMSE using Train Mean = 39.039275311505314


In [6]:
sub = pd.read_csv(r"D:\Resume Projects\Backpack Prediction Challenge\sample_submission.csv")
print('Submission shape', sub.shape)
sub['Price'] = train_mean
sub.to_csv("submission_mean.csv",index=False)
sub.head()

Submission shape (200000, 2)


,id,Price
0,300000,81.411107
1,300001,81.411107
2,300002,81.411107
3,300003,81.411107
4,300004,81.411107


Baseline 2 - Target Encode Weight Capacity - CV 38.71  
We will use RAPIDS Target Encoder to TE the numeric column weight capacity.

In [12]:
from sklearn.preprocessing import KBinsDiscretizer
from category_encoders import TargetEncoder

# Drop rows with missing values in the Price column
train_non_null = train.dropna(subset=['Price'])

TE = TargetEncoder(smoothing=20)
train_non_null['pred'] = TE.fit_transform(train_non_null['Weight Capacity (kg)'], train_non_null.Price)
s = np.sqrt(np.mean((train_non_null.Price - train_non_null.pred)**2.0))
print(f"Validation RMSE using Target Encode Weight Capacity = {s}")

Validation RMSE using Target Encode Weight Capacity = 74.70486820060027


C:\Users\omjad\AppData\Local\Temp\ipykernel_3768\929976371.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_non_null['pred'] = TE.fit_transform(train_non_null['Weight Capacity (kg)'], train_non_null.Price)


In [14]:
test = pd.read_csv(r'D:\Resume Projects\Backpack Prediction Challenge\test.csv')
sub['Price'] = TE.transform(test['Weight Capacity (kg)'])
sub.to_csv("submission_TE_weight_capacity.csv", index=False)
sub.head()

,id,Price
0,300000,20.671147
1,300001,13.564105
2,300002,11.809799
3,300003,18.477036
4,300004,9.907953
